In [1]:
#드라이브 마운트: 구글 드라이브의 데이터를 가져올 때 가져오는 라이브러리
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#필요한 모듈 import
import pandas as pd
import numpy as np
import os

In [3]:
#data 확인
for dirname, _, filenames in os.walk('/content/drive/MyDrive/MoveLens/'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/drive/MyDrive/MoveLens/genome_scores.csv
/content/drive/MyDrive/MoveLens/link.csv
/content/drive/MyDrive/MoveLens/movie.csv
/content/drive/MyDrive/MoveLens/rating.csv
/content/drive/MyDrive/MoveLens/genome_tags.csv
/content/drive/MyDrive/MoveLens/tag.csv


In [4]:
#data load
movie=pd.read_csv('/content/drive/MyDrive/MoveLens/movie.csv') 
rating=pd.read_csv('/content/drive/MyDrive/MoveLens/rating.csv')
link=pd.read_csv('/content/drive/MyDrive/MoveLens/link.csv')
tag=pd.read_csv('/content/drive/MyDrive/MoveLens/tag.csv') 
genome_tags=pd.read_csv('/content/drive/MyDrive/MoveLens/genome_tags.csv') 
genome_scores=pd.read_csv('/content/drive/MyDrive/MoveLens/genome_scores.csv')

data load 후 data 확인

In [5]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
tag.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [9]:
genome_tags.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [10]:
genome_scores.head()

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


In [11]:
#preprocessing
movie['genres'] = movie.genres.str.split('|') 
print(movie.head())
movieFeatures = movie.copy()
for index, row in movie.iterrows():
    for genre in row['genres']:
        movieFeatures.at[index, genre] = 1
movieFeatures = movieFeatures.fillna(0)
movieFeatures.head()

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                              genres  
0  [Adventure, Animation, Children, Comedy, Fantasy]  
1                     [Adventure, Children, Fantasy]  
2                                  [Comedy, Romance]  
3                           [Comedy, Drama, Romance]  
4                                           [Comedy]  


,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),[Comedy],0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#Title, year 분리
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))',expand=False) 
movie['year'] = movie.year.str.extract('(\d\d\d\d)',expand=False) 
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
movie['title'] = movie['title'].apply(lambda x: x.strip())
movie

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",NaN
2,3,Grumpier Old Men,"[Comedy, Romance]",NaN
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",NaN
4,5,Father of the Bride Part II,[Comedy],NaN
...,...,...,...,...
27273,131254,Kein Bund für's Leben,[Comedy],NaN
27274,131256,"Feuer, Eis & Dosenbier",[Comedy],NaN
27275,131258,The Pirates,[Adventure],NaN
27276,131260,Rentun Ruusu,[(no genres listed)],NaN


MoveLens data 확인

In [20]:
#data 크기 확인
print(len(rating['userId'].unique()))
print(len(rating['movieId'].unique()))

138493
26744


In [21]:
#data의 크기가 너무 커서 user-movie matrix를 만들 수 없음 => preprocessing 진행
movie_pivot=rating.pivot_table(columns='userId', index='movieId',values='rating')

ValueError: ignored

In [16]:
#1. 500개 이상 평가한 user만 filtering 진행
x=rating['userId'].value_counts()>500
print(len(x[x]))
thin_rating=rating[rating['userId'].isin(x[x].index)] 
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

7441


In [17]:
#user filtering 결과 확인
movie_pivot

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.5,4.0,4.0,5.0,4.0,NaN,3.0,4.0,5.0,4.0,...,NaN,2.5,3.5,5.0,3.0,NaN,4.0,5.0,4.0,5.0
2,NaN,NaN,3.0,NaN,3.5,NaN,2.0,NaN,5.0,NaN,...,NaN,2.5,2.5,3.0,4.0,NaN,3.0,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,1.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
5,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#2. 50개 이상 평가된 movie만 filtering 진행
y=thin_rating['movieId'].value_counts()>50
print(len(y[y]))
thin_rating=thin_rating[thin_rating['movieId'].isin(y[y].index)] 
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

9313


In [19]:
#movie filtering 결과 확인 => preprocessing 과정에서 matrix의 크기가 줄어든 것을 확인할 수 있음
movie_pivot

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.5,4.0,4.0,5.0,4.0,NaN,3.0,4.0,5.0,4.0,...,NaN,2.5,3.5,5.0,3.0,NaN,4.0,5.0,4.0,5.0
2,NaN,NaN,3.0,NaN,3.5,NaN,2.0,NaN,5.0,NaN,...,NaN,2.5,2.5,3.0,4.0,NaN,3.0,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,1.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
5,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#3. 결측치 0으로 채우기
movie_pivot.fillna(0, inplace=True)
movie_pivot

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.5,4.0,4.0,5.0,4.0,0.0,3.0,4.0,5.0,4.0,...,0.0,2.5,3.5,5.0,3.0,0.0,4.0,5.0,4.0,5.0
2,0.0,0.0,3.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,...,0.0,2.5,2.5,3.0,4.0,0.0,3.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
5,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
118997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


문제: user 24가 movie 2를 몇점으로 평가할까?

In [24]:
#Item-Item Collaborative Filtering
#필요한 module import
#csr_matrix: sparse한 matrix를 효율적으로 다루기 위한 구조
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

#item 기준으로 user들이 평가한 점수를 기반으로 NearestNeighbor 모델 생성 후 fitting 진행
movie_sparse=csr_matrix(movie_pivot) 
model=NearestNeighbors(n_neighbors=10,algorithm='brute',metric='cosine') 
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [39]:
#movie 2의 user 평가 점수
movie2_rating = movie_pivot.iloc[1,:].values.reshape(1,-1)
movie2_rating

array([[0., 0., 3., ..., 0., 0., 4.]])

In [40]:
#movie2와 비슷하게 uesr들에게 평가 받은 movie를 nearest neighbor로 찾음
distances,suggestions=model.kneighbors(movie2_rating) 
distances,suggestions

(array([[0.        , 0.22626448, 0.23797185, 0.23983944, 0.24134662,
         0.24469272, 0.2489989 , 0.25214387, 0.2534928 , 0.25541199]]),
 array([[   1,  438,  658, 1273,  326,  334,  337, 2112, 1037,  532]]))

In [30]:
#movie2와 비슷한 평가를 받은 top 10 movies
movie[movie['movieId'].isin(movie_pivot.index[suggestions[0]])]

,movieId,title,genres,year
1,2,Jumanji,"[Adventure, Children, Fantasy]",NaN
352,356,Forrest Gump,"[Comedy, Drama, Romance, War]",NaN
360,364,"Lion King, The","[Adventure, Animation, Children, Drama, Musica...",NaN
363,367,"Mask, The","[Action, Comedy, Crime, Fantasy]",NaN
476,480,Jurassic Park,"[Action, Adventure, Sci-Fi, Thriller]",NaN
580,586,Home Alone,"[Children, Comedy]",NaN
767,780,Independence Day (a.k.a. ID4),"[Action, Adventure, Sci-Fi, Thriller]",NaN
1242,1270,Back to the Future,"[Adventure, Comedy, Sci-Fi]",NaN
1528,1580,Men in Black (a.k.a. MIB),"[Action, Comedy, Sci-Fi]",NaN
2486,2571,"Matrix, The","[Action, Sci-Fi, Thriller]",NaN


In [31]:
#비슷한 평가를 받은 top 10 movie의 user 평가
#user 24가 movie 2와 비슷한 평가를 받은 영화에 대한 평점을 다음과 같이 주었으니 대략 4.28으로 예상함
movie_pivot.iloc[suggestions[0]]

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,3.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,...,0.0,2.5,2.5,3.0,4.0,0.0,3.0,0.0,0.0,4.0
480,5.0,3.0,5.0,4.0,3.5,0.0,4.0,4.0,5.0,4.5,...,4.0,3.0,3.5,5.0,5.0,4.5,4.0,4.0,0.0,5.0
780,5.0,4.0,5.0,2.0,3.0,0.0,1.0,3.0,5.0,4.0,...,3.0,1.5,3.0,3.0,5.0,5.0,3.0,2.5,4.5,0.0
1580,5.0,5.0,4.0,4.0,3.5,3.0,2.5,4.0,5.0,1.5,...,3.0,2.0,3.0,4.5,5.0,5.0,3.5,4.0,4.5,5.0
356,5.0,5.0,5.0,4.5,4.0,0.0,4.0,3.0,5.0,1.0,...,4.0,3.5,0.0,4.5,5.0,4.5,4.0,4.0,0.0,5.0
364,4.0,0.0,0.0,0.0,4.5,0.0,4.0,0.0,4.0,3.5,...,4.0,3.0,0.0,5.0,3.0,4.0,4.0,3.5,0.0,5.0
367,4.0,4.0,4.0,5.0,2.5,0.0,1.0,3.0,4.0,3.5,...,3.0,3.0,1.5,0.0,1.0,0.0,3.5,3.5,0.0,5.0
2571,5.0,5.0,5.0,5.0,3.5,1.0,4.0,3.0,5.0,3.0,...,3.0,3.5,4.0,4.0,5.0,4.5,4.0,5.0,5.0,5.0
1270,5.0,4.0,5.0,0.0,4.0,3.0,3.5,3.0,4.0,4.0,...,4.0,3.5,3.5,5.0,4.0,4.5,4.0,4.5,4.0,0.0


In [50]:
#예상 값 채우기
movie_pivot.iloc[suggestions[0, 1:]].replace(0, np.nan).mean(skipna=True)
movie_pivot_est = movie_pivot.copy()

In [51]:
movie_pivot_est.loc[2] = movie_pivot.iloc[suggestions[0,1:]].replace(0, np.nan).mean(skipna=True)
movie_pivot_est

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.500000,4.000000,4.0,5.000000,4.0,0.000000,3.000000,4.000000,5.000000,4.000000,...,0.000000,2.500000,3.500000,5.0000,3.0,0.000000,4.000000,5.000,4.0,5.000000
2,4.666667,4.285714,4.5,3.642857,3.5,2.333333,2.777778,3.285714,4.555556,3.166667,...,3.333333,2.888889,3.083333,4.3125,4.0,4.571429,3.777778,3.875,4.5,4.857143
3,0.000000,0.000000,0.0,0.000000,3.0,0.000000,2.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,3.0,5.000000,0.000000,0.000,0.0,0.000000
4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,...,1.000000,0.000000,0.000000,0.0000,4.0,0.000000,0.000000,0.000,0.0,0.000000
5,0.000000,2.000000,3.0,0.000000,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118700,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,0.000000
118900,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,0.000000
118997,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,0.000000
